Testing stuff out from the gate

In [1]:
!curl -X GET "https://api.socialdata.tools/twitter/statuses/show?id=1740192018918170738" \
-H "Accept: application/json" \
-H "Authorization: Bearer 539|UUzjs2bbjYLuc4cACbztsBRnIv7y5xHufiZYwDuoa9058562"

{"status":"error","message":"Insufficient balance"}

Creating a Function that uses the search endpoint and creates a Dataframe out of it.

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Social Data API endpoint and your API key
API_ENDPOINT = "https://api.socialdata.tools/twitter/search"
API_KEY = "574|9dLEGFBiWHWnFsZj04Tx5fflaBFJEgJqLq7PTedh7a138543"

def fetch_tweets(keywords, start_date, end_date, limit=1000):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    params = {
        "query": " OR ".join(keywords)+ " AND news",
        "type": "Top",
        "limit": limit,
    }

    response = requests.get(API_ENDPOINT, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error fetching tweets: {response.status_code} - {response.text}")

def process_tweets(tweets_data):
    processed_tweets = []
    for tweet in tweets_data['tweets']:
        processed_tweets.append({
            'created_at': tweet['tweet_created_at'],
            'text': tweet['full_text'],
            'username': tweet['user']['screen_name'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'media_url': tweet['entities']['media'][0]['media_url_https'] if 'media' in tweet['entities'] else None
        })
    return processed_tweets

# Main execution
if __name__ == "__main__":
    # Set up search parameters
    keywords = ["bitcoin", "ethereum", "cryptocurrency", "crypto"]
    start_date = datetime.now() - timedelta(days=10)  # Last 7 days
    end_date = datetime.now()

    # Fetch tweets
    raw_tweets = fetch_tweets(keywords, start_date, end_date)

    # Process tweets
    processed_tweets = process_tweets(raw_tweets)

    # Convert to DataFrame
    df = pd.DataFrame(processed_tweets)

    # Save to CSV
    csv_filename = 'cryptocurrency_tweets.csv'
    df.to_csv(csv_filename, index=False)

    print(f"Extracted {len(df)} tweets about cryptocurrency.")
    print(f"Data saved to '{csv_filename}'")

    # Display first few rows
    df.head()

Extracted 19 tweets about cryptocurrency.
Data saved to 'cryptocurrency_tweets.csv'
                    created_at  \
0  2024-08-07T19:39:34.000000Z   
1  2024-08-08T14:25:09.000000Z   
2  2024-08-08T17:51:59.000000Z   
3  2024-08-08T14:50:53.000000Z   
4  2024-08-08T20:22:06.000000Z   

                                                text         username  \
0  BREAKING NEWS 🚨\n\nImagine being able to seaml...             tari   
1  JUST IN: 🇷🇺 President Putin signs law legalizi...        BRICSinfo   
2  #Bitcoin Price Nears $58.5K for the Week as US...  cryptodataspace   
3  BREAKING:\n\nRussia legalizes crypto payments ...     GlobeEyeNews   
4  Valour Inc. Partners with Nairobi Stock Exchan...            BTCTN   

   retweet_count  favorite_count  \
0           7028            7919   
1           1590            8855   
2             11             181   
3            700            4959   
4              4              18   

                                           media_url  


In [4]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1iAopajpYNVPWFpDnBwFLTOxI0Al9JH0VxR1NY96HHv0#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


## Key Points to notice

- We cant sort by top posts cuz most of these are giveaway/promotional posts that provide us no information about the news and updates (This will be figured by putting better queries)
- We can only search 20 posts at a time. I tried using more using this endpoint [POST https://api.socialdata.tools/twitter/search/asyncJob](https://api.socialdata.tools/twitter/search/asyncJob) but it immediately drained my account balance.
- Most of the output is in Other languages too. We must filter out languages other than english.

Running Cryptobert on these datasets

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

tokenizer = BertTokenizer.from_pretrained("kk08/CryptoBERT")
model = BertForSequenceClassification.from_pretrained("kk08/CryptoBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [6]:


classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
for text in df['text']:
  result = classifier(text)
  print(result)



[{'label': 'LABEL_1', 'score': 0.9698319435119629}]
[{'label': 'LABEL_1', 'score': 0.7727152705192566}]
[{'label': 'LABEL_0', 'score': 0.6579211354255676}]
[{'label': 'LABEL_1', 'score': 0.959865152835846}]
[{'label': 'LABEL_1', 'score': 0.9642979502677917}]
[{'label': 'LABEL_1', 'score': 0.84419184923172}]
[{'label': 'LABEL_1', 'score': 0.9526029229164124}]
[{'label': 'LABEL_0', 'score': 0.921322762966156}]
[{'label': 'LABEL_1', 'score': 0.8499043583869934}]
[{'label': 'LABEL_1', 'score': 0.8901015520095825}]
[{'label': 'LABEL_1', 'score': 0.7805076837539673}]
[{'label': 'LABEL_0', 'score': 0.8959743976593018}]
[{'label': 'LABEL_0', 'score': 0.8730844855308533}]
[{'label': 'LABEL_1', 'score': 0.9497835636138916}]
[{'label': 'LABEL_1', 'score': 0.9559611678123474}]
[{'label': 'LABEL_0', 'score': 0.9050823450088501}]
[{'label': 'LABEL_1', 'score': 0.9663221836090088}]
[{'label': 'LABEL_1', 'score': 0.9266130924224854}]
[{'label': 'LABEL_1', 'score': 0.9052128791809082}]


Now Getting News using News API

In [7]:
!pip install newsapi-python

In [8]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='4670c278aa53490b99c29e290984acd4')
# /v2/everything
all_articles = newsapi.get_everything(q='bitcoin OR ethereum',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param= datetime.now()-timedelta(days=10),
                                      to=datetime.now(),
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)


all_articles

{'status': 'ok', 'totalResults': 2, 'articles': []}

In [9]:
bruh= requests.get(f"https://newsapi.org/v2/everything?q=cryptocurrency OR bitcoin OR ethereum OR blockchain&from={datetime.now()-timedelta(days=10)}&to={datetime.now}&sortBy=popularity&language=en&apiKey=4670c278aa53490b99c29e290984acd4")

In [10]:
bruh.json()

{'status': 'ok',
 'totalResults': 3426,
 'articles': [{'source': {'id': 'wired', 'name': 'Wired'},
   'author': 'Steven Levy',
   'title': "Trump's Crypto Embrace Could Be a Disaster for Bitcoin",
   'description': "At the Bitcoin 2024 conference in Nashville, Donald Trump promised the crypto community the moon. They'd better hope they don't get it.",
   'url': 'https://www.wired.com/story/donald-trump-bitcoin-reserve-promises/',
   'urlToImage': 'https://media.wired.com/photos/66ab594d0c0cc4819f595db4/191:100/w_1280,c_limit/073024_Crypto%20get%20rich%20quick.jpg',
   'publishedAt': '2024-08-02T13:00:00Z',
   'content': 'Donald Trump is an unlikely crypto ally. The power of bitcoin, embodied in Satoshi Nakamotos founding document, is that it frees participants from murky assessments of trust, instead relying on the b… [+4248 chars]'},
  {'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'Lucas Ropek',
   'title': 'Kamala Harris Wants to Get Closer to Crypto Bros, If They’ll L

In [11]:
df_news = pd.json_normalize(bruh.json()['articles'])
df_news


,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Steven Levy,Trump's Crypto Embrace Could Be a Disaster for...,"At the Bitcoin 2024 conference in Nashville, D...",https://www.wired.com/story/donald-trump-bitco...,https://media.wired.com/photos/66ab594d0c0cc48...,2024-08-02T13:00:00Z,Donald Trump is an unlikely crypto ally. The p...,wired,Wired
1,Lucas Ropek,Kamala Harris Wants to Get Closer to Crypto Br...,Cryptocurrency has become the latest flashpoin...,https://gizmodo.com/kamala-harris-wants-to-get...,https://gizmodo.com/app/uploads/2019/05/oqakha...,2024-07-30T13:30:20Z,"Over the past week, Kamala Harris has made it ...",None,Gizmodo.com
2,Lucas Ropek,This Could Be the Beginning of the End of Trum...,The Donald chose the wrong time to get into web3.,https://gizmodo.com/this-could-be-the-beginnin...,https://gizmodo.com/app/uploads/2024/08/Trump_...,2024-08-05T21:35:52Z,Donald Trump’s honeymoon with the cryptocurren...,None,Gizmodo.com
3,Lucas Ropek,Akon’s Multi-Billion Dollar Crypto City in Afr...,"Will the singer's $6 billion metropolis, a ""re...",https://gizmodo.com/akons-multi-billion-dollar...,https://gizmodo.com/app/uploads/2024/08/akon_c...,2024-08-02T19:20:00Z,Rich people have become obsessed with building...,None,Gizmodo.com
4,jsor@businessinsider.com (Jennifer Sor),Stock market today: Dow jumps 285 points as tr...,The Bank of Japan suggested on Wednesday it wo...,https://markets.businessinsider.com/news/stock...,https://i.insider.com/66b378f5a5247369a3ea253d...,2024-08-07T13:45:50Z,US stocks jumped on Wednesday after the Bank o...,business-insider,Business Insider
...,...,...,...,...,...,...,...,...,...
95,kcloonan@insider.com (Kelly Cloonan),Stock market today: Indexes tank as data paint...,Stock indexes tanked Thursday as new data show...,https://markets.businessinsider.com/news/stock...,https://i.insider.com/66abe9a4b4912df3ae185614...,2024-08-01T20:26:15Z,US stock indexes saw a deep sell-off on Thursd...,business-insider,Business Insider
96,kcloonan@insider.com (Kelly Cloonan),Stock market today: Indexes dip after Meta ear...,The losses come despite strong Meta earnings a...,https://markets.businessinsider.com/news/stock...,https://i.insider.com/66ab9601aab37011780c5bdc...,2024-08-01T14:37:44Z,Indexes slid Thursday amid a strong earnings r...,business-insider,Business Insider
97,AndroidGuys,Aviator 1xBet – Your Chance for Big Winnings i...,Play Aviator on 1xBet for big winnings. Regist...,https://androidguys.com/promoted-news/aviator-...,https://i0.wp.com/androidguys.com/wp-content/u...,2024-08-02T15:42:36Z,The 1xBet betting platform is among the widely...,None,AndroidGuys
98,Dade Hayes,New Paramount Bidder Apex Capital Trust Enters...,With three weeks remaining in the “go-shop” pe...,http://deadline.com/2024/07/new-paramount-bid-...,https://deadline.com/wp-content/uploads/2023/0...,2024-07-31T19:02:24Z,With three weeks remaining in the “go-shop” pe...,None,Deadline


In [12]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_news)

https://docs.google.com/spreadsheets/d/1fwaF-5dwiNPodDnZGubANq1ATqaYPDtA-ppDWUhbtpU#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


### Keyword and Langchain

In [13]:
!pip install groq langchain-groq langchain langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [14]:
import os
from google.colab import userdata


This one is using the groq libary directly

In [15]:
from groq import Groq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
client = Groq(api_key=userdata.get('GROQ_API_KEY'))
completion = client.chat.completions.create(
    model="llama3-groq-70b-8192-tool-use-preview",
    messages=[
        {
            "role": "system",
            "content": "JSON You are a bot whose job is to take Crypto News put it in the category of the top 50 cryptocurrencies of the world, the list is:\nBitcoin\nEthereum\nTether USD\nBNB\nSolana\nUSDC\nXRP\nLido Staked Ether\nToncoin\nDogecoin\nCardano\nTRON\nWrapped liquid staked Ether 2.0\nWrapped BTC\nAvalanche\nShiba Inu\nWrapped Ether\nPolkadot\nBitcoin Cash\nChainlink\nDai\nUniswap\nLitecoin\nPolygon\nBinance-Peg BSC-USD\nKaspa\nWrapped eETH\nInternet Computer (DFINITY)\nPEPE\nUSDe\nEthereum Classic\nMonero\nPancakeSwap\nAptos\nNEAR Protocol\nImmutable X\nFetch.AI\nOKB\nStacks\nFilecoin\nBittensor\nStellar\nFirst Digital USD\nMantle\nHedera\nVeChain\nWhiteBIT Coin\nRender Token\nEnergySwap\nMaker\n\nUnless a new Cryptocurrency is mentioned (with its code) always put the articles in these list. If the news is general and not applicable to any cryptocurrency You can also mark the Article as general.\nExample:\n\nDogecoin (DOGE) currently holds a market capitalization of $19  billion. Despite experiencing a 14% price surge over the past two weeks, the cryptocurrency remains significantly below its all-time hi… [+4205 chars]\n\nCryptocurrencies: Dogecoin\n\nExperts predict that Toshi, one of the most popular Base meme coins, could go up by 10x this year because of the recent surge in meme coins.\nThis article explores the recent developments, features, … [+5097 chars]\n\nCryptocurrencies: TOSHI\n\nExperts predict a big surge for Pepe (PEPE), leading many investors to believe now is the best time to buy in.\nThe cryptocurrency market is on the rise. Market volume has jumped by 62%, and the Fear… [+5122 chars]\n\nCryptocurrencies: PEPE\n\nWith $45.5 million in corporate contributions, American cryptocurrency exchange Coinbase is the largest donor to Fairshake: a newly-minted super PAC focused solely on installing political candidates … [+10647 chars]\n\nCryptocurrencies: General\n\n"
        },
        {
            "role": "user",
            "content": "With Bitcoin (BTC) surging and the market exhibiting greed according to the Fear and Greed Index, it’s an opportune time to consider diversifying your portfolio with altcoin investments.\nWhile meme … [+6891 chars]"
        }
    ],
    temperature=0.5,
    max_tokens=1024,
    top_p=0.65,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)

print(completion.choices[0].message.content)


{"cryptocurrencies": ["Bitcoin"]}


In [16]:
top_cryptos = [
    "Bitcoin", "Ethereum", "Tether USD", "BNB", "Solana", "USDC", "XRP", "Lido Staked Ether",
    "Toncoin", "Dogecoin", "Cardano", "TRON", "Wrapped liquid staked Ether 2.0", "Wrapped BTC",
    "Avalanche", "Shiba Inu", "Wrapped Ether", "Polkadot", "Bitcoin Cash", "Chainlink", "Dai",
    "Uniswap", "Litecoin", "Polygon", "Binance-Peg BSC-USD", "Kaspa", "Wrapped eETH",
    "Internet Computer (DFINITY)", "PEPE", "USDe", "Ethereum Classic", "Monero", "PancakeSwap",
    "Aptos", "NEAR Protocol", "Immutable X", "Fetch.AI", "OKB", "Stacks", "Filecoin", "Bittensor",
    "Stellar", "First Digital USD", "Mantle", "Hedera", "VeChain", "WhiteBIT Coin", "Render Token",
    "EnergySwap", "Maker",'BRETT', 'The Doge NFT', 'DEGEN', 'TOSHI', 'doginme', 'Normie',
    'OmniCat', 'Marvin', 'RealGOAT', 'Mister Miggles', 'Basenji',
    'UPSIDE DOWN MEME', 'SPX6900', '$MFER', 'ChompCoin', 'SKOP Token',
    'Marso.Tech', 'Keyboard Cat', 'higher', 'donotfomoew', 'Pepe',
    'Base God', 'Crash', 'BORED', 'Roost Coin', 'BUILD', 'FomoBullClub',
    'Noggles', 'All Street Bets', 'WASSIE', 'DINO', 'Moby', 'Mamba',
    'Shoobadookie', 'Based Street Bets', 'Ski Mask Dog', 'AEROBUD',
    'Fungi', 'FOMO_BASE', 'ROCKY', 'Rug World Assets', 'BlockChainPeople',
    'Apedinbase', 'GameStop on Base', 'Based Shiba Inu', 'Misser',
    'Father Of Meme: Origin', 'Katt Daddy', 'Heroes of memes', 'DERP'
]

prompt = PromptTemplate(
    input_variables=["text"],
    template="""
        Given the following text:
        {text}

        Identify any cryptocurrencies mentioned in the text and categorize them as 'crypto'. If no cryptocurrencies are found, categorize the text as 'general'.

        The response should be a list of dictionaries with the following format:
        [
            {{
                "name": "Bitcoin",
                "category": "crypto"
            }},
            {{
                "name": "Ethereum",
                "category": "crypto"
            }},
            {{
                "name": "This text is not about any specific cryptocurrencies",
                "category": "general"
            }}
        ]
    """
)

In [17]:
from langchain_groq import ChatGroq


In [18]:
from typing import List, Tuple
def crypto_categorizer(text: str) -> List[dict]:
    llm = ChatGroq(groq_api_key = userdata.get('GROQ_API_KEY'),model = "llama3-groq-70b-8192-tool-use-preview")
    #chain = LLMChain(prompt=prompt, llm=llm)
    chain = prompt | llm | parser
    result = chain.invoke(text)
    return eval(result)

### Using langchain to make our thing

Prompt Template

In [42]:
def get_sentiment(news_articles):
  """
  Given the news article, returns the Sentiment Analysis Score on the Text """
  classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
  classy=classifier(news_articles)
  if classy[0]['label']=="LABEL_1":
    sentiment="Positive"
  else:
    sentiment="Negative"
  prompt = PromptTemplate(
    input_variables=["news_articles","sentiment"],
    template="""

        You are an expert in Cryptocurrency and have tons of experience in the analysis and speculations in the crypto market. You are given the follwing news article:
        {news_articles}

        We are getting the following sentiment from this article:
        {sentiment}

        In your expert knowledge in this field, explain why there could be this given sentiment:
    """
)
  llm = ChatGroq(groq_api_key = userdata.get('GROQ_API_KEY'),model = "llama3-groq-70b-8192-tool-use-preview")
  chain = prompt | llm | parser
  result = chain.invoke({"news_articles":news_articles,"sentiment":sentiment})


  return f"{sentiment} {classy[0]['score']} \n Here is why: \n {result}"


In [25]:
text = "JUST IN: 🇷🇺 President Putin signs law legalizing cryptocurrency mining in Russia. https://t.co/GqpcmFTqDR"
categories = crypto_categorizer(text)
for category in categories:
    print(f"{category['name']} - {category['category']}")

This text is not about any specific cryptocurrencies - general


Explaining the Sentiment using Groq.

In [43]:
query = """
🚨BREAKING: MAJOR BLACK SWAN EVENT IS HAPPENING!!

XRP and CTF Token suddenly are priced at $198.53 for minutes on CMC.

Trading has been Halted on Multiple Crypto exchanges.

CryptoTradingFund Token (https://t.co/Vw93uDoegt) on the XRP Ledger has also begun Skyrocketing, Has the BULLRUN STARTED?!"""

get_sentiment(query)

'Negative 0.8959743976593018 \n Here is why: \n The sentiment of the article appears to be negative for several reasons:\n\n1. Black Swan Event: The term "Black Swan Event" is often associated with unexpected and catastrophic events in the financial world. This implies that the sudden price spike and trading halt could have a negative impact on investors and the market as a whole.\n\n2. Price Spike and Trading Halt: A sudden price spike can be alarming, especially when it\'s followed by a trading halt. This can cause uncertainty and fear among investors, leading to a negative sentiment.\n\n3. Impact on Exchanges: The fact that trading has been halted on multiple crypto exchanges suggests that there might be underlying issues or concerns that need to be addressed. This could lead to further market volatility and potential losses for investors, contributing to the negative sentiment.\n\n4. Uncertainty and Lack of Clarity: The article does not provide clear reasons for the price spike and